# Caderno de teste - Métricas

A primeira parte do caderno é um conjunto de testes com a biblioteca evaluate/trec_eval.

Em seguida, o caderno contém uma implementação própria para o cálculo das métricas precisão (P@k), recall (R@k), MRR (MRR@k) e nDCG (nDCG@k). A ideia é ter um código um pouco mais legível, mesmo que ineficiente.

Fontes:

- https://huggingface.co/spaces/evaluate-metric/trec_eval

- https://github.com/joaopalotti/trectools

## Teste 1 - Execução simples testando uma query

Vamos supor que o qrel da query 0 indica 3 documentos, doc_1, doc_2 e doc_3, cujas relevâncias são 3, 2, 1.

O sistema de busca retornou, nessa ordem, doc_2, doc_1, doc_10, doc_11, doc_12:

In [1]:
from evaluate import load
trec_eval = load("trec_eval")
 
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [3, 2, 1]
    }
run = {
    "query": [0, 0, 0, 0, 0], # QUERY ID
    "q0": ["q0", "q0", "q0", "q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1", "doc_10", "doc_11", "doc_12"], # DOCUMENT_ID
    "rank": [1, 2, 3, 3, 4], # RANKING DO DOCUMENTO
    "score": [2, 3, 0, 0, 0], # SCORE DO DOCUMENTO
    "system": ["test", "test", "test", "test", "test"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

0.4
0.8174935137996165


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


Agora, vamos ver o que ocorre se tirarmos os três documentos não relevantes (não pode mudar nada):

In [2]:
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [3, 2, 1]
    }
run = {
    "query": [0, 0], # QUERY ID
    "q0": ["q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1"], # DOCUMENT_ID
    "rank": [0, 1], # RANKING DO DOCUMENTO
    "score": [1.5, 1.2], # SCORE DO DOCUMENTO
    "system": ["test", "test"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


0.4
0.8174935137996165


Checando o efeito do score no qrels (só pode mudar o ndcg, mas a precisão tem que continuar a mesma):

In [3]:
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [10, 9, 8]
    }
run = {
    "query": [0, 0], # QUERY ID
    "q0": ["q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1"], # DOCUMENT_ID
    "rank": [0, 1], # RANKING DO DOCUMENTO
    "score": [1.5, 1.2], # SCORE DO DOCUMENTO
    "system": ["test", "test"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


0.4
0.777975983841851


Agora vamos alterar novamente o score no qrel, mas para 30, 20, 10 (mantém a mesma proporção que 3, 2, 1):

In [4]:
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [30, 20, 10]
    }
run = {
    "query": [0, 0], # QUERY ID
    "q0": ["q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1"], # DOCUMENT_ID
    "rank": [0, 1], # RANKING DO DOCUMENTO
    "score": [1.5, 1.2], # SCORE DO DOCUMENTO
    "system": ["test", "test"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

0.4
0.8174935137996167


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


Vamos ver se o score no run faz algum efeito:

In [5]:
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [3, 2, 1]
    }
run = {
    "query": [0, 0], # QUERY ID
    "q0": ["q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1"], # DOCUMENT_ID
    "rank": [0, 1], # RANKING DO DOCUMENTO
    "score": [0, 1000], # SCORE DO DOCUMENTO
    "system": ["test", "test"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


0.4
0.8174935137996165


As conclusões aqui:

- O score no qrel fazem diferença pro nDCG. Uma relação de 2/1 no score do qrel equivale a uma relação de 6/2. A relação entre os scores no qrel parece ser multiplicativa.

- O score no run parece não fazer diferença para o nDCG. Mesmo mudando a ordem (colocando um score mais alto para quem está mais atrás no ranking) não faz diferença nos valores.

## Teste 2 - Execução testando uma query e dois sistemas

Vamos supor que o qrel da query 0 indica 3 documentos, doc_1, doc_2 e doc_3, cujas relevâncias são 3, 2, 1.

O sistema 1 retornou, nessa ordem, doc_2, doc_1.

O sistema 2 retornou apenas doc_3.

In [6]:
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [3, 2, 1]
    }
run = {
    "query": [0, 0, 0], # QUERY ID
    "q0": ["q0", "q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1", "doc_3"], # DOCUMENT_ID
    "rank": [0, 1, 0], # RANKING DO DOCUMENTO
    "score": [2, 1, 2], # SCORE DO DOCUMENTO
    "system": ["sistema1", "sistema1", "sistema2"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

0.6
0.9224945116765986


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


O resultado não é o que queremos e dá pra ver isso na precisão.

Estou interpretando "system" como um sistema, então quero o resultado por sistema. 

O que eu esperava aqui é ter uma precisão/ndcg para o sistema 1 e uma precisão/ndcg para o sistema 2. Vamos tentar separar o run em dois, run_sistema_1 e run_sistema_2:

In [7]:
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [3, 2, 1]
    }
run_sistema_1 = {
    "query": [0, 0], # QUERY ID
    "q0": ["q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1"], # DOCUMENT_ID
    "rank": [0, 1], # RANKING DO DOCUMENTO
    "score": [2, 1], # SCORE DO DOCUMENTO
    "system": ["sistema1", "sistema1"] # SISTEMA
}
 
run_sistema_2 = {
    "query": [0], # QUERY ID
    "q0": ["q0"], # LITERAL q0
    "docid": ["doc_3"], # DOCUMENT_ID
    "rank": [0], # RANKING DO DOCUMENTO
    "score": [2], # SCORE DO DOCUMENTO
    "system": ["sistema2"] # SISTEMA
}

try:
    results = trec_eval.compute(predictions=[run_sistema_1, run_sistema_2], references=[qrel])
    print(results['P@5'])
    print(results['NDCG@5'])
except:
    print('Essa abordagem não dá certo')


Essa abordagem não dá certo


O jeito parece ser separar e rodar duas vezes:

In [8]:
qrel = {
    "query": [0, 0, 0],
    "q0": ["q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3"],
    "rel": [3, 2, 1]
    }
run_sistema_1 = {
    "query": [0, 0], # QUERY ID
    "q0": ["q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1"], # DOCUMENT_ID
    "rank": [0, 1], # RANKING DO DOCUMENTO
    "score": [2, 1], # SCORE DO DOCUMENTO
    "system": ["sistema1", "sistema1"] # SISTEMA
}
 
run_sistema_2 = {
    "query": [0], # QUERY ID
    "q0": ["q0"], # LITERAL q0
    "docid": ["doc_3"], # DOCUMENT_ID
    "rank": [0], # RANKING DO DOCUMENTO
    "score": [2], # SCORE DO DOCUMENTO
    "system": ["sistema2"] # SISTEMA
}

results_sistema_1 = trec_eval.compute(predictions=[run_sistema_1], references=[qrel])
results_sistema_2 = trec_eval.compute(predictions=[run_sistema_2], references=[qrel])
print(results_sistema_1['P@5'])
print(results_sistema_1['NDCG@5'])
print('*'*30)
print(results_sistema_2['P@5'])
print(results_sistema_2['NDCG@5'])


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


0.4
0.8174935137996165
******************************
0.2
0.21000199575396408


## Teste 3: agregando queries diferentes para um sistema

Agora que sabemos como a ferramenta trata sistemas diferentes (é necessário executar separadamente), vamos ver como é o tratamento com mais de uma query. Vamos testar 3 queries:

O qrel de cada query está assim:

- query 0
    - docs: doc_1, doc_2, doc_3
    - relevância: 3, 2, 1

- query 1
    - docs: doc_1, doc_5, doc_6
    - relevância: 3, 2, 1

- query 2
    - docs: doc_3
    - relevância: 3

O sistema de busca retornou:

- query 0:
    - docs: doc_1, doc_2 (P@5 = 2/5 = 0.4)
- query 1:
    - docs: doc_5 (P@5 = 1/5 = 0.2)
- query 2:
    - docs: não retornou nada (P@5 = 0)

Como a query 2 não retornou nada, vou fazer o primeiro teste sem passar ela:

In [9]:
qrel = {
    "query": [0, 0, 0, 1, 1, 1, 2],
    "q0": ["q0", "q0", "q0", "q0", "q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3", "doc_1", "doc_5", "doc_6", "doc_3"],
    "rel": [3, 2, 1, 3, 2, 1, 3]
    }
run = {
    "query": [0, 0, 1], # QUERY ID
    "q0": ["q0", "q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1", "doc_5"], # DOCUMENT_ID
    "rank": [0, 1, 0], # RANKING DO DOCUMENTO
    "score": [2, 1, 2], # SCORE DO DOCUMENTO
    "system": ["test", "test", "test"] # SISTEMA
}
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

0.30000000000000004
0.6187487526537724


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


A média dos P@5 deve ser de (0.4 + 0.2 + 0)/3 = 0.2.

Como não passamos a query 2, ele desconsiderou-a da métrica. Assim, mesmo que não tenha resultados, é necessário informá-la de alguma forma:

In [10]:
qrel = {
    "query": [0, 0, 0, 1, 1, 1, 2],
    "q0": ["q0", "q0", "q0", "q0", "q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3", "doc_1", "doc_5", "doc_6", "doc_3"],
    "rel": [3, 2, 1, 3, 2, 1, 3]
    }
run = {
    "query": [0, 0, 1, 2], # QUERY ID
    "q0": ["q0", "q0", "q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1", "doc_5", ""], # DOCUMENT_ID
    "rank": [0, 1, 0, -1], # RANKING DO DOCUMENTO
    "score": [2, 1, 2, -1], # SCORE DO DOCUMENTO
    "system": ["test", "test", "test", "test"] # SISTEMA
}
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

0.20000000000000004
0.4124991684358483


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


Passando vazio no docid e -1 no rank/score funcionou. O mesmo ocorre passando alguma string inexistente no docid e qualquer outro número no rank/score:

In [11]:
qrel = {
    "query": [0, 0, 0, 1, 1, 1, 2],
    "q0": ["q0", "q0", "q0", "q0", "q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3", "doc_1", "doc_5", "doc_6", "doc_3"],
    "rel": [3, 2, 1, 3, 2, 1, 3]
    }
run = {
    "query": [0, 0, 1, 2], # QUERY ID
    "q0": ["q0", "q0", "q0", "q0"], # LITERAL q0
    "docid": ["doc_2", "doc_1", "doc_5", "XXXXXXX"], # DOCUMENT_ID
    "rank": [0, 1, 0, 0], # RANKING DO DOCUMENTO
    "score": [2, 1, 2, 0], # SCORE DO DOCUMENTO
    "system": ["test", "test", "test", "test"] # SISTEMA
}
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print(results['P@5'])
print(results['NDCG@5'])

0.20000000000000004
0.4124991684358483


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


## Teste cálculo manual do nDCG para apenas uma query

In [12]:
import math

# Vamos considerar que a entrada é no seguinte formato:
# doc_retornados = ["doc_a", "doc_b", "doc_c" etc]. A posição indica a ordem
# Para facilitar, vamos considerar que doc_relevantes é um dict assim:
# {"doc_x": score, "doc_y": score etc}
def dcg(doc_retornados, doc_relevantes, k=None, debug=True, aproximacao_trec_eval=False):
    dcg = 0
    doc_retornados = doc_retornados if k is None else doc_retornados[:k]
    for rank, doc_id in enumerate(doc_retornados, 1):
        # Relevância do documento
        rel = doc_relevantes.get(doc_id, 0)
        # Cálculo do ganho. Aproximação trec_eval usa diretamente a relevância
        gain = (2**(rel) - 1) if not aproximacao_trec_eval else rel
        dcg_i = gain/(math.log(rank + 1, 2))
        dcg += dcg_i
        if debug:
            print(doc_id, rank, dcg_i)

    if debug:
        print('\n')
    return dcg

def idcg(doc_retornados, doc_relevantes, k=None, debug=True, aproximacao_trec_eval=False):
    # Cria uma lista de tuplas (doc_id, relevância, posição original na lista de retornados)
    # para todos os documentos relevantes
    # A posição original só é usada para desempate, portanto, ela segue a ordem de doc_retornados
    docs_com_relevancia = [
        (doc, 
        doc_relevantes.get(doc, 0), # Nem precisava de get, pois certamente existe
        doc_retornados.index(doc) if doc in doc_retornados else len(doc_retornados))
        for doc in doc_relevantes.keys()
    ]

    # Ordena os documentos primeiro pela relevância (decrescente) e depois pela posição original (crescente)
    # Isso garante que, em caso de empate na relevância, o documento que apareceu primeiro em doc_retornados ganhe
    docs_ordenados = sorted(docs_com_relevancia, key=lambda x: (-x[1], x[2]))

    # Extrai apenas os doc_ids da lista ordenada
    doc_retornados_ideal = [doc[0] for doc in docs_ordenados]

    return dcg(doc_retornados_ideal, doc_relevantes, k, debug, aproximacao_trec_eval)

def ndcg(doc_retornados, doc_relevantes, k=None, debug=True, aproximacao_trec_eval=False):
    return dcg(doc_retornados, doc_relevantes, k, debug, aproximacao_trec_eval) / idcg(doc_retornados, doc_relevantes, k, debug, aproximacao_trec_eval)

doc_retornados = ["A", "B", "C", "D", "E"]
doc_relevantes = {"A": 2, "B": 3, "D": 1, "E": 2}
ndcg(doc_retornados, doc_relevantes)

A 1 3.0
B 2 4.416508275000202
C 3 0.0
D 4 0.43067655807339306
E 5 1.1605584217036249


B 1 7.0
A 2 1.8927892607143721
E 3 1.5
D 4 0.43067655807339306




0.8322420383257692

Vamos testar essa função com os resultados obtidos do trec_eval:

In [13]:
doc_retornados = ["doc_2", "doc_1", "doc_10", "doc_11", "doc_12"]
doc_relevantes = {"doc_1": 3, "doc_2": 2, "doc_3": 1}
print(ndcg(doc_retornados, doc_relevantes, k=None, debug=False, aproximacao_trec_eval=False))
print(ndcg(doc_retornados, doc_relevantes, k=None, debug=False, aproximacao_trec_eval=True))
print('Esperado: 0.8174')


0.7895959410076381
0.8174935137996168
Esperado: 0.8174


In [14]:
doc_retornados = ["doc_3"]
doc_relevantes = {"doc_1": 3, "doc_2": 2, "doc_3": 1}
print(ndcg(doc_retornados, doc_relevantes, k=None, debug=False, aproximacao_trec_eval=False))
print(ndcg(doc_retornados, doc_relevantes, k=None, debug=False, aproximacao_trec_eval=True))
print('Esperado: 0.210')


0.10646464774659968
0.2100019957539641
Esperado: 0.210


Testando cálculo manual do nDCG@k:

Vamos considerar que o qrels tem mais do que 5 resultados, apenas para checarmos como fica o nDCG@5 e o nDCG@10.

Vamos considerar também que a lista de resultados tem mais do que 5 elementos.

Nesse caso, devem dar resultados diferentes:

In [15]:

qrel = {
    "query": [0, 0, 0, 0, 0, 0],
    "q0": ["q0", "q0", "q0", "q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3", "doc_4", "doc_5", "doc_6"],
    "rel": [3, 2, 1, 3, 2, 1]
    }


run = {
    "query": [0, 0, 0, 0, 0], # QUERY ID
    "q0": ["q0", "q0", "q0", "q0", "q0"], # LITERAL q0
    "docid": ["doc_1", "A", "B", "C", "D"], # DOCUMENT_ID
    "rank": [0, 1, 2, 3, 4], # RANKING DO DOCUMENTO
    "score": [6, 5, 4, 3, 2], # SCORE DO DOCUMENTO
    "system": ["test", "test", "test", "test", "test"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print('TREC_EVAL:')
print(f"nDCG@5: {results['NDCG@5']}")
print(f"nDCG@10: {results['NDCG@10']}")


doc_retornados = ["doc_1", "A", "B", "C", "D"]
doc_relevantes = {"doc_1": 3, "doc_2": 2, "doc_3": 1, "doc_4": 3, "doc_5": 2, "doc_6": 1}
print('IMPLEMENTAÇÃO:')
print(f"nDCG@5: {ndcg(doc_retornados, doc_relevantes, k=5, debug=False, aproximacao_trec_eval=True)}")
print(f"nDCG@10: {ndcg(doc_retornados, doc_relevantes, k=10, debug=False, aproximacao_trec_eval=True)}")

TREC_EVAL:
nDCG@5: 0.42010951172205624
nDCG@10: 0.40014926254662797
IMPLEMENTAÇÃO:
nDCG@5: 0.4201095117220563
nDCG@10: 0.40014926254662797


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


In [16]:

qrel = {
    "query": [0, 0, 0, 0, 0, 0],
    "q0": ["q0", "q0", "q0", "q0", "q0", "q0"],
    "docid": ["doc_1", "doc_2", "doc_3", "doc_4", "doc_5", "doc_6"],
    "rel": [3, 2, 1, 3, 2, 1]
    }


run = {
    "query": [0, 0, 0, 0, 0], # QUERY ID
    "q0": ["q0", "q0", "q0", "q0", "q0"], # LITERAL q0
    "docid": ["doc_1", "A", "B", "C", "doc_3"], # DOCUMENT_ID
    "rank": [0, 1, 2, 3, 4], # RANKING DO DOCUMENTO
    "score": [6, 5, 4, 3, 2], # SCORE DO DOCUMENTO
    "system": ["test", "test", "test", "test", "test"] # SISTEMA
}
 
 
results = trec_eval.compute(predictions=[run], references=[qrel])
print('TREC_EVAL:')
print(f"nDCG@5: {results['NDCG@5']}")
print(f"nDCG@10: {results['NDCG@10']}")


doc_retornados = ["doc_1", "A", "B", "C", "doc_3"]
doc_relevantes = {"doc_1": 3, "doc_2": 2, "doc_3": 1, "doc_4": 3, "doc_5": 2, "doc_6": 1}
print('IMPLEMENTAÇÃO:')
print(f"nDCG@5: {ndcg(doc_retornados, doc_relevantes, k=5, debug=False, aproximacao_trec_eval=True)}")
print(f"nDCG@10: {ndcg(doc_retornados, doc_relevantes, k=10, debug=False, aproximacao_trec_eval=True)}")

TREC_EVAL:
nDCG@5: 0.4742830263739263
nDCG@10: 0.4517488843896262
IMPLEMENTAÇÃO:
nDCG@5: 0.47428302637392633
nDCG@10: 0.45174888438962624


C:\Users\caris\anaconda3\Lib\site-packages\trectools\trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


## Implementação de uma lib para facilitar o cálculo das métricas

A ideia aqui é implementar um conjunto de funções que receberá dataframes Pandas para o resultado e para o qrels e retornará um conjunto de métricas (MRR, Precision, Recall, nDCG).

Como é para uso pessoal e em poucos testes, minha inteção é prezar pela legibilidade em detrimento da eficiência.

In [17]:
import pandas as pd

def precisao_recall(docs_retornados, docs_relevantes, k=None):
    """
        Dado um conjunto de documentos retornados e de documentos relevantes,
        calcula a precisão e o recall em k para uma query.

        docs_retornados -- Objeto Series contendo os documentos retornados ordenados
        docs_relevantes -- Objeto Series contendo os documentos relevantes
        k -- No cálculo da precisão e recall, indica até que posição dos documentos
                retornados deve ser considerada.

        Se k = None, toda a lista de documentos retornados é considerada. Se k != None,
        considera apenas os k'éssimos primeiros documentos retornados.
        Para o cálculo da precisão, se k = None, considera no denominador o total de documentos retornados
    """
    if k is None:
        k = len(docs_retornados)
    docs_retornados_em_k = docs_retornados[:k]
    docs_retornados_em_k_relevantes = set(docs_retornados_em_k) & set(docs_relevantes)

    precisao = len(docs_retornados_em_k_relevantes)/max(k, 1)
    recall = len(docs_retornados_em_k_relevantes)/len(docs_relevantes)

    return precisao, recall

def mrr(docs_retornados, docs_relevantes, k=None):
    """
        Calcula o MRR@k (Mean Reciprocal Rank) para uma query.

        docs_retornados -- Objeto Series contendo os documentos retornados ordenados
        docs_relevantes -- Objeto Series contendo os documentos relevantes
        k -- Indica até que posição dos documentos retornados deve ser considerada.
    """
    if k is None:
        k = len(docs_retornados)

    mrr_score = 0.0
    set_docs_relevantes = set(docs_relevantes)
    for i in range(min(k, len(docs_retornados))):
        if docs_retornados.iloc[i] in set_docs_relevantes:
            mrr_score = 1.0 / (i+1) # Soma com 1 pois a posição começa em 1 e i começa em 0.
            break
    return mrr_score

def dcg(doc_retornados, doc_relevantes, k=None, debug=True, aproximacao_trec_eval=False):
    """
        Calcula DCG@k para uma query.

        doc_retornados -- É uma lista de keys de documentos. A posição do documento na lista
            indica a ordem
        docs_relevantes -- É um dict cuja chave é a key e um documento relevante e o valor
            é o seu score
        k -- Indica até que posição dos documentos retornados deve ser considerada.
        debug -- Indica se é pra imprimir o cálculo intermediário
        aproximacao_trec_eval -- Se True, usa a relevância como Linear. Se False, usa
            como 2^(rel)
    """
    dcg = 0
    doc_retornados = doc_retornados if k is None else doc_retornados[:k]
    for rank, doc_id in enumerate(doc_retornados, 1):
        # Relevância do documento
        rel = doc_relevantes.get(doc_id, 0)
        # Cálculo do ganho. Aproximação trec_eval usa diretamente a relevância
        gain = (2**(rel) - 1) if not aproximacao_trec_eval else rel
        dcg_i = gain/(math.log(rank + 1, 2))
        dcg += dcg_i
        if debug:
            print(doc_id, rank, dcg_i)

    if debug:
        print('\n')
    return dcg

def idcg(doc_retornados, doc_relevantes, k=None, debug=True, aproximacao_trec_eval=False):
    """
        Calcula iDCG@k para uma query.

        doc_retornados -- É uma lista de keys de documentos. A posição do documento na lista
            indica a ordem
        docs_relevantes -- É um dict cuja chave é a key e um documento relevante e o valor
            é o seu score
        k -- Indica até que posição dos documentos retornados deve ser considerada.
        debug -- Indica se é pra imprimir o cálculo intermediário
        aproximacao_trec_eval -- Se True, usa a relevância como Linear. Se False, usa
            como 2^(rel)
    """
    # Cria uma lista de tuplas (doc_id, relevância, posição original na lista de retornados)
    # para todos os documentos relevantes
    # A posição original só é usada para desempate, portanto, ela segue a ordem de doc_retornados
    docs_com_relevancia = [
        (doc, 
        doc_relevantes.get(doc, 0), # Nem precisava de get, pois certamente existe
        doc_retornados.index(doc) if doc in doc_retornados else len(doc_retornados))
        for doc in doc_relevantes.keys()
    ]

    # Ordena os documentos primeiro pela relevância (decrescente) e depois pela posição original (crescente)
    # Isso garante que, em caso de empate na relevância, o documento que apareceu primeiro em doc_retornados ganhe
    docs_ordenados = sorted(docs_com_relevancia, key=lambda x: (-x[1], x[2]))

    # Extrai apenas os doc_ids da lista ordenada
    doc_retornados_ideal = [doc[0] for doc in docs_ordenados]

    return dcg(doc_retornados_ideal, doc_relevantes, k, debug, aproximacao_trec_eval)

def ndcg(resultado_pesquisa, qrels, col_resultado_doc_key, col_qrels_doc_key, col_qrels_score, k=None, debug=True, aproximacao_trec_eval=False):
    """
        Calcula o nDCG@k para uma query

        resultado_pesquisa -- DataFrame Pandas com o resultado da pesquisa. Considera que
            o DataFrame está ordenado de acordo com os documentos retornados
        qrels -- DataFrame Pandas com o qrels

        col_resultado_doc_key -- indica a KEY do documento retornado.
        col_qrels_doc_key -- indica a KEY de um documento associado a query.
        col_qrels_score -- indica a relevância do documento para aquela query. Quanto maior, mais relevante.

        k -- Indica até que posição dos documentos retornados deve ser considerada.
        debug -- Indica se é pra imprimir o cálculo intermediário
        aproximacao_trec_eval -- Se True, usa a relevância como Linear. Se False, usa
            como 2^(rel)
    """
    # Converte os pandas para lista de doc_retornados e dict de doc_relevantes por score:
    doc_retornados = resultado_pesquisa[col_resultado_doc_key].tolist()
    doc_relevantes = dict(zip(qrels[col_qrels_doc_key], qrels[col_qrels_score]))

    return dcg(doc_retornados, doc_relevantes, k, debug, aproximacao_trec_eval) / idcg(doc_retornados, doc_relevantes, k, debug, aproximacao_trec_eval)

def metricas(resultado_pesquisa, qrels, 
             col_resultado_query_key="QUERY_KEY",
             col_resultado_doc_key="DOC_KEY",
             col_resultado_rank="RANK",
             col_qrels_query_key="QUERY_KEY",
             col_qrels_doc_key="DOC_KEY",
             col_qrels_score="SCORE",
             k=[5, 10, 50], debug=False, aproximacao_trec_eval=False):
    """
        Calcula um conjunto de métricas para um resultado de pesquisa e um conjunto qrels.
        resultado_pesquisa -- DataFrame Pandas contendo o resultado das pesquisas.
        qrels -- DataFrame Pandas contendo o qrels

        Os parâmetros col_resultado_xxxx referem-se a nomes de colunas no DataFrame resultado_pesquisa:

        col_resultado_query_key -- indica a KEY da query.
        col_resultado_doc_key -- indica a KEY do documento retornado.
        col_resultado_rank -- indica a posição do documento retornado.

        Os parâmetros col_qrels_xxxx referem-se a nomes de colunas no DataFrame qrels:

        col_qrels_query_key -- indica a KEY da query que será testada.
        col_qrels_doc_key -- indica a KEY de um documento associado a query.
        col_qrels_score -- indica a relevância do documento para aquela query. Quanto maior, mais relevante.
    """
    # Remove do qrels os resultados cujo score é 0
    qrels = qrels[qrels[col_qrels_score] > 0]

    # Extrai as queries que devem ser analisadas. Se tiver query no resultado que não 
    # está no qrels, ela não será avaliada.
    query_keys = qrels.QUERY_KEY.unique()

    precisao_em_k = {valor_k: [0]*len(query_keys) for valor_k in k}
    recall_em_k = {valor_k: [0]*len(query_keys) for valor_k in k}
    mrr_em_k = {valor_k: [0]*len(query_keys) for valor_k in k}
    ndcg_em_k = {valor_k: [0]*len(query_keys) for valor_k in k}

    for i_q_key, q_key in enumerate(query_keys):
        # Extrai o resultado e o qrels para a query que irá ser analisada
        resultado_para_query = resultado_pesquisa[resultado_pesquisa[col_resultado_query_key] == q_key]
        qrels_para_query = qrels[qrels[col_qrels_query_key] == q_key]
        
        # Pega os docs retornados (ordenados de acordo com a posição deles na pesquisa, em ordem crescente - Rank 1 para cima)
        # e os docs relevantes.
        resultado_para_query = resultado_para_query.sort_values(by=col_resultado_rank)
        docs_retornados = resultado_para_query[col_resultado_doc_key]
        docs_relevantes = qrels_para_query[col_qrels_doc_key]

        for valor_k in k:
            p_em_k, r_em_k = precisao_recall(docs_retornados, docs_relevantes, valor_k)
            precisao_em_k[valor_k][i_q_key] = p_em_k
            recall_em_k[valor_k][i_q_key] = r_em_k
            mrr_em_k[valor_k][i_q_key] = mrr(docs_retornados, docs_relevantes, valor_k)
            ndcg_em_k[valor_k][i_q_key] = ndcg(resultado_para_query, qrels_para_query, col_resultado_doc_key, col_qrels_doc_key, col_qrels_score, valor_k, debug, aproximacao_trec_eval)

    pd_metricas = pd.DataFrame({'QUERY_KEY': query_keys})

    # Insere as métricas na ordem: precisão, recall, MRR, nDCG:
    for valor_k in k:
        pd_metricas[f'P@{valor_k}'] = precisao_em_k[valor_k]
    for valor_k in k:
        pd_metricas[f'R@{valor_k}'] = recall_em_k[valor_k]
    for valor_k in k:
        pd_metricas[f'MRR@{valor_k}'] = mrr_em_k[valor_k]
    for valor_k in k:
        pd_metricas[f'nDCG@{valor_k}'] = ndcg_em_k[valor_k]

    return pd_metricas

## Testes com dados fictícios

Refaz os testes anteriores. Para isso, separa cada teste como um query_key diferente.

In [18]:
# DataFrame qrels
QRELS_QUERY_KEY = [0, 0, 0,
                   1, 1, 1,
                   2, 2, 2,
                   3, 3, 3]
QRELS_DOC_KEY = ["doc_1", "doc_2", "doc_3",
                 "doc_1", "doc_2", "doc_3",
                 "doc_1", "doc_2", "doc_3",
                 "doc_1", "doc_2", "doc_3"]
QRELS_SCORE = [3, 2, 1,
               10, 9, 8,
               3, 2, 1,
               3, 2, 1]

# DataFrame resultado
RESULTADO_QUERY_KEY = [0, 0, 0, 0, 0,
                       1, 1,
                       2, 2,
                       3]
RESULTADO_DOC_KEY = ["doc_2", "doc_1", "doc_10", "doc_11", "doc_12",
                     "doc_2", "doc_1",
                     "doc_2", "doc_1",
                     "doc_3"]
RESULTADO_RANK = [1, 2, 3, 4, 5,
                  1, 2,
                  1, 2,
                  1]

# DataFrames
resultado = pd.DataFrame({
        "QUERY_KEY": RESULTADO_QUERY_KEY,
        "DOC_KEY": RESULTADO_DOC_KEY,
        "RANK": RESULTADO_RANK
})
qrels = pd.DataFrame({
        "QUERY_KEY": QRELS_QUERY_KEY,
        "DOC_KEY": QRELS_DOC_KEY,
        "SCORE": QRELS_SCORE
})

# Métricas
pd_metricas = metricas(resultado, qrels, aproximacao_trec_eval=True)

display(pd_metricas)

,QUERY_KEY,P@5,P@10,P@50,R@5,R@10,R@50,MRR@5,MRR@10,MRR@50,nDCG@5,nDCG@10,nDCG@50
0,0,0.4,0.2,0.04,0.666667,0.666667,0.666667,1.0,1.0,1.0,0.817494,0.817494,0.817494
1,1,0.4,0.2,0.04,0.666667,0.666667,0.666667,1.0,1.0,1.0,0.777976,0.777976,0.777976
2,2,0.4,0.2,0.04,0.666667,0.666667,0.666667,1.0,1.0,1.0,0.817494,0.817494,0.817494
3,3,0.2,0.1,0.02,0.333333,0.333333,0.333333,1.0,1.0,1.0,0.210002,0.210002,0.210002


## Testes com os dados da pesquisa de jurisprudência selecionada

In [19]:
import pandas as pd

PASTA_DADOS = './dados/'
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'
PASTA_RESULTADO_PESQUISA_SOLR = f'{PASTA_DADOS}outputs/2_pesquisa_queries_na_base_atual/'

qrels = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')
resultado_pesq_solr = pd.read_csv(f'{PASTA_RESULTADO_PESQUISA_SOLR}resultado_solr_pesquisa_original.csv', sep='|')

# Seleciona só o resultado do \select
resultado_select = resultado_pesq_solr[resultado_pesq_solr.ENGINE == 'SOLR_PESQUISA_ORIGINAL_select']

In [20]:
pd_metricas = metricas(resultado_select, qrels)

In [21]:
pd_metricas[0:50].describe()

,QUERY_KEY,P@5,P@10,P@50,R@5,R@10,R@50,MRR@5,MRR@10,MRR@50,nDCG@5,nDCG@10,nDCG@50
count,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,25.50000,0.228000,0.224000,0.121200,0.090891,0.181985,0.496675,0.325333,0.351127,0.363872,0.195865,0.220778,0.388672
std,14.57738,0.302399,0.235225,0.065827,0.117575,0.185708,0.294862,0.405325,0.387716,0.376717,0.289161,0.256444,0.242699
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.25000,0.000000,0.000000,0.080000,0.000000,0.000000,0.283333,0.000000,0.000000,0.080420,0.000000,0.000000,0.205387
50%,25.50000,0.200000,0.200000,0.120000,0.069048,0.133333,0.461538,0.200000,0.200000,0.200000,0.074950,0.140229,0.379109
75%,37.75000,0.350000,0.400000,0.180000,0.122727,0.302198,0.750000,0.500000,0.500000,0.500000,0.228105,0.306173,0.538699
max,50.00000,1.000000,0.900000,0.220000,0.384615,0.642857,1.000000,1.000000,1.000000,1.000000,1.000000,0.913356,0.945387


In [22]:
pd_metricas[50:100].describe()

,QUERY_KEY,P@5,P@10,P@50,R@5,R@10,R@50,MRR@5,MRR@10,MRR@50,nDCG@5,nDCG@10,nDCG@50
count,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,75.50000,0.400000,0.248000,0.060400,0.166995,0.206648,0.253084,0.789000,0.794833,0.794833,0.531603,0.460424,0.472605
std,14.57738,0.291373,0.214989,0.057355,0.124864,0.184427,0.250287,0.377328,0.365834,0.365834,0.284397,0.236777,0.229619
min,51.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,63.25000,0.200000,0.100000,0.020000,0.083333,0.083333,0.083333,0.625000,0.625000,0.625000,0.406728,0.388781,0.400098
50%,75.50000,0.400000,0.200000,0.040000,0.153846,0.153846,0.153846,1.000000,1.000000,1.000000,0.553146,0.470047,0.490327
75%,87.75000,0.600000,0.300000,0.080000,0.230769,0.282468,0.326923,1.000000,1.000000,1.000000,0.716849,0.592450,0.603464
max,100.00000,1.000000,0.900000,0.200000,0.500000,0.900000,1.000000,1.000000,1.000000,1.000000,1.000000,0.939877,0.931057


In [23]:
pd_metricas[100:150].describe()

,QUERY_KEY,P@5,P@10,P@50,R@5,R@10,R@50,MRR@5,MRR@10,MRR@50,nDCG@5,nDCG@10,nDCG@50
count,50.00000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,125.50000,0.00800,0.004000,0.001200,0.003636,0.003636,0.005303,0.040000,0.040000,0.041000,0.014843,0.011077,0.012476
std,14.57738,0.03959,0.019795,0.004798,0.017995,0.017995,0.021222,0.197949,0.197949,0.197869,0.073739,0.056005,0.056083
min,101.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,113.25000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,125.50000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,137.75000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,150.00000,0.20000,0.100000,0.020000,0.090909,0.090909,0.090909,1.000000,1.000000,1.000000,0.403015,0.333743,0.329363
